In [1]:
# Testing ollama embedding using phi-3.5-mini https://ollama.com/blog/embedding-models
# compares direct query and HyDE query

# steps to install phi-3.5-mini on ollama
# 1. install ollama
# 2. run >ollama pull phi3.5

# phi-3.5-mini embedding query results are wrong from both direct query and HyDE query!!!
# the results is ['5', '0', '4', '1', '2', '3'] from direct query and ['5', '1', '4', '0', '2', '3'] from HyDE query
# mxbai-embed-large performs for embedding.

import ollama
import chromadb

# embedding_mode = "phi3.5"
# embedding_mode = "mxbai-embed-large"
embedding_mode = "snowflake-arctic-embed"

documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()
collection = client.get_or_create_collection(name="docs"+embedding_mode)

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model=embedding_mode, prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [2]:
collection.peek(), collection.count()

({'ids': ['0', '1', '2', '3', '4', '5'],
  'embeddings': array([[ 1.38336241e-01, -6.37730837e-01,  3.30837298e-04, ...,
          -2.99776822e-01,  3.60927694e-02, -7.95960069e-01],
         [ 7.44450539e-02, -1.40076292e+00,  5.44786692e-01, ...,
          -5.72953939e-01, -5.76398671e-01, -1.68894932e-01],
         [ 1.29173249e-01, -9.00350213e-01,  5.86453438e-01, ...,
          -1.37285376e+00,  1.15311090e-02, -3.43049407e-01],
         [ 3.79785180e-01, -8.54738593e-01,  3.19993168e-01, ...,
          -2.88958490e-01,  8.52232799e-02, -4.31492180e-02],
         [ 1.67640358e-01, -1.53667367e+00, -2.61566699e-01, ...,
          -1.06726420e+00,  4.18261617e-01, -1.23311877e-01],
         [-3.58988106e-01, -1.83536172e+00,  3.73736739e-01, ...,
          -9.80534434e-01, -1.74474776e-01,  1.26849085e-01]]),
  'metadatas': [None, None, None, None, None, None],
  'documents': ["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
 

In [3]:
collection.get(ids=["0"])

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [None],
 'documents': ["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels"],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [4]:
# an example prompt
prompt = "What animals are llamas related to?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model=embedding_mode
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=1
)
data = results['documents'][0][0]

In [5]:
results

{'ids': [['0']],
 'distances': [[156.59153747558594]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [6]:
show_distances = collection.query(response["embedding"], include=["distances"], n_results=6)
show_distances

{'ids': [['0', '4', '1', '3', '5', '2']],
 'distances': [[156.59153747558594,
   236.48777770996094,
   240.87335205078125,
   264.4229736328125,
   278.2029724121094,
   283.7362976074219]],
 'metadatas': None,
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'included': ['distances']}

In [7]:
# the result should be the first document: Llamas are members of the camelid family meaning ...

# use HyDE to create a tempate answer from the question.
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

hyde_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user queries succinctly."),
    ("user", "Question: {question}")
])
output_parser=StrOutputParser()
llm = ChatOllama(model="phi3.5", temperature=0)
chain = hyde_prompt | llm | output_parser

converted_q = chain.invoke({'question': prompt})
converted_q

'Llamas are most closely related to camels and other members of the Camelidae family, such as alpacas, guanacos, and vicuñas. They share a common ancestor that lived around 35 million years ago in North America before migrating into South America where they evolved independently for millions more years.'

In [8]:
# query documents by the converted_q
converted_response = ollama.embeddings( prompt=converted_q, model=embedding_mode)
converted_results = collection.query(query_embeddings=[converted_response["embedding"]], n_results=1)
converted_results

{'ids': [['0']],
 'distances': [[98.01392364501953]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [9]:
show_converted_distances = collection.query(converted_response["embedding"], include=["distances"], n_results=6)
show_converted_distances

{'ids': [['0', '1', '4', '5', '2', '3']],
 'distances': [[98.01392364501953,
   186.42498779296875,
   226.71197509765625,
   245.62966918945312,
   302.3124694824219,
   304.7037048339844]],
 'metadatas': None,
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'included': ['distances']}

In [10]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="phi3.5",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

Llamas are indeed quite closely related to two other species within the Camelidae family, namely vicuñas and true camels (which include dromedary camels and Bactrian camels). Here's a more detailed response:

- Vicunas: Llamas share evolutionary roots with vicuñas. Both llamas and vicuñas have been domesticated for their wool, which is known to be warm yet lightweight—ideal in high altitude regions where these animals are native such as the Andes mountains of South America.
  
- Camels: Llamas share a closer genetic link with camels compared to other wild or semi-domesticated relatives like guanacos and alpaca, which belong to different subfamilies within the broader superfamily Artiodactyla (even-toed ungulates). True Camelus species—dromedaries found in North Africa/Middle East regions and Bactrian camels native from Central Asia – have diverged significantly over time. Nevertheless, llamas still share certain morphological features like their long necks, humps on the shoulders for c

In [11]:
# generate a response combining the prompt and data retrieved without extra information
output = ollama.generate(
  model="phi3.5",
  prompt=f"Please generate a summary based on below data succinctly without additional information. Here is the data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

Llamas, as members of the camelid family, share close relations with other camelids such as vicuñas and various species of true camels (e.g., dromedary and Bactrian camels). These animals possess similar adaptations for life in arid environments, like long eyelashes, dense fur, and humps or a broad chest to store fat reserves which are essential survival traits shared among them.
